# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [02:03<00:00, 123.78s/it]


In [4]:
len(deals)

20

In [5]:
deals[5].describe()

'Title: Garvee 3-Ton Triple Air Bag Jack for $86 + free shipping\nDetails: Apply coupon code DEAL527 for an extra savings of $51. Buy Now at Garvee\nFeatures: \nURL: https://www.dealnews.com/Garvee-3-Ton-Triple-Air-Bag-Jack-for-86-free-shipping/21743635.html?iref=rss-c238'

In [5]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [6]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [7]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Auto Accessory Deals at eBay: Up to 95% off + free shipping
Details: There are some massive lows to be had here, we found plenty of items that are double or more elsewhere today. We've picked out this 1,000-lbs. Steel Engine Cradle for Ford for $38.99 (better than half off.) Shop Now at eBay
Features: 
URL: https://www.dealnews.com/Auto-Accessory-Deals-at-eBay-Up-to

In [8]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [9]:
result = get_recommendations()

In [10]:
len(result.deals)

5

In [11]:
result.deals[1]

Deal(product_description="This back-up camera is specifically tailored for the 2011-2013 Jeep Grand Cherokee and Dodge Durango models. Featuring a wide 130-degree angle for enhanced visibility, this camera improves safety by providing a clearer view of what's behind your vehicle. Its compact design ensures easy installation, allowing you to enhance your vehicle's safety features without hassle. Priced at $17, this back-up camera is an affordable way to add extra safety to your ride.", price=17.0, url='https://www.dealnews.com/Back-Up-Camera-for-2011-2013-Jeep-Grand-Cherokee-Dodge-Durango-for-17-free-shipping/21744186.html?iref=rss-c238')

In [12]:
from agents.scanner_agent import ScannerAgent

In [13]:
agent = ScannerAgent()
result = agent.scan()

In [14]:
result

DealSelection(deals=[Deal(product_description='This 5-Gallon Jerry Can is ideal for storing and transporting fuel or other liquids safely. Constructed from durable materials, it is designed to prevent spills and leaks. The can comes with a sturdy handle for easy lifting and pouring, making it perfect for both recreational use and emergency situations. At a price of $31, this jerry can represents a great value for those who need reliable liquid storage.', price=31.0, url='https://www.dealnews.com/5-Gallon-Jerry-Can-for-31-free-shipping/21741296.html?iref=rss-c238'), Deal(product_description='The 3-Ton Triple Air Bag Jack from Garvee offers superior lifting capability for a variety of vehicles. It features a triple air bag design to provide stability and ease of use. The jack is perfect for both professional mechanics and DIY enthusiasts, allowing you to lift vehicles quickly and safely. Priced at $86, this jack includes free shipping for added convenience.', price=86.0, url='https://www